# Ejercicio 1

Definimos las variables aleatorias $T_{i}$ : " Tiempo en segundos que transcurre entre la $i-ésima$ y la $ i+1-ésima$ llamada al servicio " y generamos muestras aleatorias para las mismas.

$T_{i} \sim \varepsilon(\frac{1}{4}) $

In [1]:
import numpy as np
from numpy.random import RandomState

In [2]:
cantidad_solicitudes = 100000
tiempos_entre_llamadas = RandomState(5).exponential(scale=4, size=cantidad_solicitudes)

# tiempos_acumulados almacena el instante en el que llega cada solicitud
tiempos_acumulados = []
tiempos_acumulados.append(tiempos_entre_llamadas[0])
for i in range(cantidad_solicitudes - 1):
    tiempos_acumulados.append(tiempos_acumulados[i] + tiempos_entre_llamadas[i+1])

## Arquitectura con dos bases de datos 

Dado que cada solicitud es procesada por una base de datos u otra de acuerdo a una distribución Bernoulli con parámetro $p = 0.6$ independientemente de las demás solicitudes, podemos aplicar la propiedad de adelgazamiento para Procesos de Poisson. En este sentido, analizamos cada base de datos por separado. 

In [3]:
def calcular_tiempos_espera(horarios_llegada, demora, cantidad_solicitudes):
    horario_base_liberada = horarios_llegada[0] + demora[0]
    tiempo_espera = []

    #la primera solicitud no debe esperar
    tiempo_espera.append(0)
    for i in range(cantidad_solicitudes - 1):
        if horarios_llegada[i+1] < horario_base_liberada:
            tiempo_espera.append(horario_base_liberada - horarios_llegada[i+1])
            horario_base_liberada += demora[i+1] 

        else:
            tiempo_espera.append(0)
            horario_base_liberada = horarios_llegada[i+1] + demora[i+1] 
    
    return tiempo_espera            

In [4]:
p = 0.6
eleccion = np.asarray(RandomState(5).binomial(1, p, cantidad_solicitudes))
tiempos_llegada_b1 = []
tiempos_llegada_b2 = []

for i in range(cantidad_solicitudes):
    if eleccion[i] == 1:
        tiempos_llegada_b1.append(tiempos_acumulados[i])
    else:
        tiempos_llegada_b2.append(tiempos_acumulados[i])
        
cantidad_solicitudes_b1 = len(tiempos_llegada_b1)
cantidad_solicitudes_b2 = len(tiempos_llegada_b2)
demora_b1 = RandomState(5).exponential(scale=0.7, size=cantidad_solicitudes_b1)
demora_b2 = RandomState(5).exponential(scale=1, size=cantidad_solicitudes_b2)

tiempos_espera_b1 = calcular_tiempos_espera(tiempos_llegada_b1, demora_b1, cantidad_solicitudes_b1)        
tiempos_espera_b2 = calcular_tiempos_espera(tiempos_llegada_b2, demora_b2, cantidad_solicitudes_b2)

In [5]:
print("El tiempo medio de espera para la primera arquitectura es de {:.3f} segundos".format(np.mean(tiempos_espera_b1 + tiempos_espera_b2)))

El tiempo medio de espera para la primera arquitectura es de 0.088 segundos


In [6]:
print("Un {:.2f} % de las solicitudes no esperaron para ser atendidas".format(np.sum(np.asarray(tiempos_espera_b1 + tiempos_espera_b2) == 0) * 100 / cantidad_solicitudes))

Un 89.47 % de las solicitudes no esperaron para ser atendidas


In [7]:
print("El tiempo medio de demora para cada solicitud es de {:.3f} segundos".format(np.sum(tiempos_espera_b1 + tiempos_espera_b2 + list(demora_b1) + list(demora_b2)) / cantidad_solicitudes))

El tiempo medio de demora para cada solicitud es de 0.912 segundos


Vale la pena resaltar que el resultado obtenido pareciera correcto. Definiendo $X_{i}$ : " Tiempo en segundos necesario para procesar la la $i-ésima$ solicitud sin tener en cuenta el tiempo de espera" se puede calcular $E[X_{i}]$ por la propiedad de esperanza total.

$E[X_{i}] = 0.6 \times 0.7 + 0.4 \times 1 = 0.82$

Considerando además el tiempo medio de espera simulado: 0.088, uno esperaría un resultado similar a 0.82 + 0.088 = 0.908.  

## Arquitectura con una sola base de datos

In [8]:
demora = RandomState(5).exponential(scale=0.8, size=cantidad_solicitudes)

In [9]:
tiempo_espera = calcular_tiempos_espera(tiempos_acumulados, demora, cantidad_solicitudes)

In [10]:
print("El tiempo medio de espera para la segunda arquitectura es de {:.3f} segundos".format(np.mean(tiempo_espera)))

El tiempo medio de espera para la segunda arquitectura es de 0.166 segundos


In [11]:
print("Un {:.2f} % de las solicitudes no esperaron para ser atendidas".format(np.sum(np.asarray(tiempo_espera) == 0) * 100 / cantidad_solicitudes))

Un 80.13 % de las solicitudes no esperaron para ser atendidas


In [12]:
print("El tiempo medio de demora para cada solicitud es de {:.3f} segundos".format(np.sum(tiempo_espera + list(demora)) / cantidad_solicitudes))

El tiempo medio de demora para cada solicitud es de 0.968 segundos


## Conclusión

En conclusión, recomendaremos que la emprese utilice la segunda opción, debido a que el tiempo medio de demora para cada solicitud de la misma es sólamente un 6.14 % más lento. A pesar de los errores cometidos en los cálculos que deducen este resultado, creemos que es lo suficientemente significativo como para poder asegurar que la segunda opción es la más conveniente. 

In [13]:
#Calculos Auxiliares
(0.968 / 0.912 - 1) * 100

6.140350877192979